In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

In [ ]:
from math import pi, sqrt, exp
from scipy.signal import convolve
import scipy.cluster.hierarchy as sch
def gauss(n=11,sigma=1):
    r = range(-int(n/2),int(n/2)+1)
    return [1 / (sigma * sqrt(2*pi)) * exp(-float(x)**2/(2*sigma**2)) for x in r]

In [ ]:
from sklearn.decomposition import PCA

# spikes

In [ ]:
st = pd.read_csv('/mnt/c/data/side/allen/spikes_to_plot_60s.csv', index_col=0).reset_index(drop=True)

In [ ]:
_st = st.query('72.5 < t < 80.5')

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
lss = []
for i, row in _st.iterrows():
    x = row['t']
    y = row['n']
    ls = LineString([(x, y), (x, y+1)])
    lss.append(ls)

In [ ]:
mls = gp.make_like(gp.merge_LineStrings(lss), drawbox)
mls = gp.scale_like(mls, drawbox, preserve_aspect_ratio=False)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.geometry(mls)

# tolerance=0.5
# sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')
# sk.vpype('linemerge --tolerance 0.2mm linesort')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/0261_allen_spikes_8s.svg')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.geometry(mls)

for tolerance in [0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2,]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
    sk.vpype('linesort')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/0186_allen_spikes_8s_connected.svg')

# smoothed spikes

In [ ]:
st = pd.read_csv('/mnt/c/data/side/allen/spikes_to_plot_60s.csv', index_col=0).reset_index(drop=True)

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
t_start = st['t'].min()
t_end = st['t'].max()
binsize = 0.001
bins = np.arange(t_start, t_end, binsize)

binned_frs = []
for n, group in st.groupby('n'):
    counts, _ = np.histogram(group['t'], bins=bins)
    binned_frs.append(counts)
bfrs = np.stack(binned_frs)    
kernel = np.array(gauss(n=500, sigma=25)).reshape(1, -1)

In [ ]:
sfrs = convolve(bfrs, kernel, 'same')

In [ ]:
scale=10

In [ ]:
lss = []
for i,row in enumerate(sfrs):
    x = np.arange(len(row)) 
    y = row*scale+i
    ls = LineString(np.stack((x,y)).T)
    lss.append(ls)

In [ ]:
mls = gp.make_like(gp.merge_LineStrings(lss), drawbox)
mls = gp.scale_like(mls, drawbox, preserve_aspect_ratio=False)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
sk.geometry(mls)

# tolerance=0.5
# sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')
# sk.vpype('linesimplify --tolerance 0.1 linesimplify --tolerance 0.1 linesort')
sk.display()

## dim reduction

In [ ]:
st = pd.read_parquet('/mnt/c/data/side/allen/spikes_to_plot_10m.parquet')

In [ ]:
st = st.query('t < 90')

In [ ]:
# make page
paper_size = '17x11 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
t_start = st['t'].min()
t_end = st['t'].max()
binsize = 0.005
bins = np.arange(t_start, t_end, binsize)

groups = st.groupby('n')
n_units = groups.ngroups
bfrs = np.zeros((len(bins)-1, n_units), dtype=int)

for i, (n, group) in enumerate(groups):
    counts, _ = np.histogram(group['t'], bins=bins)
    bfrs[:, i] = counts

In [ ]:
kernel = np.array(gauss(n=50, sigma=5)).reshape(1, -1)

In [ ]:
sfrs = convolve(bfrs, kernel, 'same')

In [ ]:
pca = PCA()
pcs = pca.fit_transform(sfrs)

In [ ]:
time_index = pd.to_timedelta(bins[:-1], unit='s')

In [ ]:
df = pd.DataFrame(pcs, index=time_index)

In [ ]:
rsdf = df

In [ ]:
scale=1

In [ ]:
n_comp_to_plot = 30
lss = []
for i,row in enumerate(rsdf.iloc[:, :n_comp_to_plot].T.values):
    x = np.arange(len(row)) 
    y = row*scale+i
    ls = LineString(np.stack((x,y)).T)
    lss.append(ls)

In [ ]:
mls = gp.make_like(gp.merge_LineStrings(lss), drawbox)
mls = gp.scale_like(mls, drawbox, preserve_aspect_ratio=False)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
sk.geometry(mls)

# tolerance=0.5
# sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')
# sk.vpype('linesimplify --tolerance 0.1 linesimplify --tolerance 0.1 linesort')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/0180_smoothed_allen_spikes.svg')

# spike triangulation craziness

In [ ]:
pts = []
for i, row in st.iterrows():
    x = row['t']
    y = row['n']
    pts.append(Point(x,y))

In [ ]:
pts = MultiPoint(pts)

In [ ]:
tris = so.triangulate(pts, edges=True)

In [ ]:
tris = gp.make_like(gp.merge_LineStrings(tris), drawbox)

In [ ]:
tris = gp.scale_like(tris, drawbox, preserve_aspect_ratio=False)

In [ ]:
tribs = tris.buffer(-0.1).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
sk.geometry(tribs)
for tolerance in [0.1, 0.2, 0.3, 0.4]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm linesimplify --tolerance {tolerance}mm')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/0189_delauney_spikes_merged.svg')

# ca2

In [ ]:
# make page
paper_size = '17x11 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
dff = pd.read_csv('/mnt/c/data/side/allen/traces.csv')

In [ ]:
dff = dff.iloc[:, 1:]

In [ ]:
df = dff.T

dt = pd.to_timedelta(df.index.astype(float), unit='s')

df.index = dt

ds = df.resample('500ms').mean()

dsbl = ds - ds.median(axis=0)

In [ ]:
60 * 30

In [ ]:
start_time = '30s'
stop_time = '1530s'
traces = dsbl.loc[start_time:stop_time].T.values

In [ ]:
scale = 0.01

In [ ]:
lss = []
for i,row in enumerate(traces):
    x = np.arange(len(row)) 
    y = row*scale+i
    ls = LineString(np.stack((x,y)).T)
    lss.append(ls)

In [ ]:
mls = gp.make_like(gp.merge_LineStrings(lss), drawbox)
mls = gp.scale_like(mls, drawbox, preserve_aspect_ratio=False)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.geometry(mls)

for tolerance in [0.1, 0.1]:
    sk.vpype(f'linesimplify --tolerance {tolerance}mm')
sk.vpype('linesort')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/0187_ca2.svg')